<img src="images\crisil_logo.png" align="right" border="0"><br>



# Capacitación en Python 07 - Análisis exploratorio de datos

En el siguiente cuaderno se presentan herramientas del análisis exploratorio de datos. Lea atentamente el cuaderno y corra el código en cada celda para visualizar su salida.

Los temas a tratar son:
 
 1. Introducción
 2. Frecuencias observadas y distribuciones empíricas
 3. Medidas de localización
 4. Medidas de dispersión
 

---
## Introducción

El análisis exploratorio de datos (EDA por sus siglas en inglés) es una manera de analizar datos resumiendo sus características básicas, generalmente mediante métodos visuales. Fue desarrollado por John W. Tukey, que lo define como

>"Procedimientos para analizar datos, técnicas para interpretar los resultados de dichos procedimientos, formas de 
planificar la recopilación de datos para hacer su análisis más fácil, más preciso o más exacto, y toda la maquinaria y resultados de las estadísticas (matemáticas) que se aplican al análisis de datos."

<img src="images\john_tukey.png" width='300' align="center" border="0"><br>

En este cuaderno se sigue en gran medida el Capítulo 3 del libro "Credit Risk Analytics", de B. Baesens, D. Rösch y H. Scheule. Con el fin de demostrar algunas herramientas del análisis exploratorio se utiliza el dataset 'mortgage', que se puede descargar de la página oficial del libro. 

El conjunto de datos reporta observaciones del origen y rendimiento de 50000 prestatarios hipotecarios residenciales de EE.UU. durante 60 períodos. Los períodos han sido desidentificados. Como en el mundo real, los préstamos pueden originarse antes del inicio del período de observación. Las observaciones del préstamo pueden ser censuradas a medida que los préstamos maduran o los prestatarios refinancian. El conjunto de datos es una selección aleatoria de datos de nivel de préstamos hipotecarios, recopilados de carteras subyacentes de valores residenciales estadounidenses respaldados por hipotecas (RMBS por sus siglas en inglés), de carteras de titulización de valores proporcionadas por International Financial Research (www.internationalfinancialresearch.org).   

In [ ]:
#importaciones necesarias.
import os # para manejar rutas y funciones del sistema operativo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn #permite generar algunas figuras más etéticas de manera más fácil que mpl 
import matplotlib as mpl 

%matplotlib inline
sn.set() #fija el formato de seaborn para las figuras

In [ ]:
nombre_dataset='mortgage.csv'
nombre_carpeta='datasets'
ruta_relativa=os.path.join(nombre_carpeta,nombre_dataset)
mortgage_data = pd.read_csv(ruta_relativa)
mortgage_data.head() #primer vistazo al dataset

In [ ]:
mortgage_data.columns #columnas del dataset

---
## Frecuencias observadas y distribuciones empíricas

Considere que tenemos como punto de partida una muestra con $n$ observaciones para una variable $X$. Para cada observación medimos un valor específico, representado por $x_{1},\cdots, x_{n}$, que se denomina _datos sin procesar_. La variable puede ser discreta o continua, pero agrupada en clases. Luego denotamos los valores o números de clase por $a_{1},\cdots,a_{k}$ y contamos los números absolutos de ocurrencia de cada valor o número de clase por:

$$ 
h_{j} = h(a_{j})
$$

y las frecuencias relativas por:

$$ 
f_{j} = \dfrac{h_{j}}{n}
$$

Obviamente, se sostiene que $\sum\limits_{j=1}^{k} h_{j} = n$ y $\sum\limits_{j=1}^{k}f_{j} = 1$. Además, definimos la frecuencia acumulada absoluta y relativa $H(x)$ y $F(x)$ para cada valor $x$ como el número de frecuencia relativa de valores que es como máximo igual a $x$

$$ \begin{array}{c}
H(x)  =  \sum\limits_{a_{j}\leq x} h(a_{j})\\
F(x)  =  \dfrac{H(x)}{n}
\end{array}
$$

Podemos calcular frecuencias utilizando la función `value _count()` de `pandas`. Por ejemplo, para la columna `default_ time` del conjunto de datos de hipotecas:

In [ ]:
mortgage_data.default_time.unique() # default es 1 , no default 0

In [ ]:
frequencies = mortgage_data.default_time.value_counts()

In [ ]:
n = len(mortgage_data) #número total de observaciones es la longitud de la columna
relative_frequencies = frequencies / n #defino mis frecuencias relativas

In [ ]:
print(f"El número total de observaciones es {n}")
print('='*45)
print(f"Frequencias:\n{frequencies}")
print('='*45)
print(sum(frequencies)) # verifico que la suma de ocurrencias sea el total de observaciones


In [ ]:
print(f"Frecuencias relativas:\n{relative_frequencies}")
print("="*45)
print(sum(relative_frequencies)) #verifico que la suma sea 1

A continuación, podemos calcular histogramas, que grafican las frecuencias absolutas (o relativas) para valores o clases de variables y la función de distribución acumulativa empírica (CDF por sus siglas en inglés) para el puntaje FICO de la variable (`FICO_orig_time`).

In [ ]:
plt.subplots(figsize = (6, 4.5))
plt.hist(mortgage_data.FICO_orig_time, bins=50)
plt.xlabel('FICO_orig_time')
plt.ylabel('Frecuencia absoluta')

In [ ]:
plt.subplots()
plt.hist(mortgage_data.FICO_orig_time, density = True, bins=50) # density = True me lleva a la frecuencia relativa
plt.xlabel('FICO_orig_time')
plt.ylabel('Frecuencia relativa')

In [ ]:
plt.subplots()
plt.hist(mortgage_data.FICO_orig_time, density = True, cumulative = True, bins=50) #prestar atención a los parámetros
plt.ylabel('CDF')
plt.xlabel('FICO_orig_time')

Alternativamente, se puede definir explicitamente la función de distribución acumulada.

In [ ]:
#creo nuevo df con los valores de FICO y su frecuecia
stats_df = mortgage_data.groupby("FICO_orig_time")["FICO_orig_time"].agg("count") #todavía no es un df  
stats_df = stats_df.pipe(pd.DataFrame) # utilizo pipe() para transformarlo en un DataFrame
stats_df = stats_df.rename(columns = {'FICO_orig_time': 'Frecuencia'}) #nombre sus columas correctamente
stats_df['pdf'] = stats_df['Frecuencia'] / sum(stats_df['Frecuencia']) #creo columna de funcion dist. probabilidad
stats_df['cdf'] = stats_df['pdf'].cumsum() #creo columna de función de distribución acumulativa
stats_df.reset_index(inplace=True) 
stats_df.tail() #reviso el final del df, para ver si cdf termina en 1 como debería

In [ ]:
plt.subplots()
plt.plot(stats_df.FICO_orig_time, stats_df.cdf)
plt.ylabel('CDF')
plt.xlabel('FICO_orig_time')

---
## Medidas de localización

Además, o como alternativa a la descripción de toda la distribución, a menudo informamos medidas que describen los datos de manera similar. Estas medidas dan información numérica sobre la localización de la distribución, su dispersión y su forma, y *generalmente* se denominan _momentos_. Se utilizan comúnmente tres medidas para la ubicación: la __media__, la __mediana__ y la __moda__.

Supongamos que tenemos $n$ valores $x_{1},\cdots,x_{n}$; entonces la media es:

$$
\bar{x} = \dfrac{1}{n} \sum\limits_{i=1}^{n} x_{i}
$$

La mediana requiere al menos una escala ordinal (es decir, ranking de valores). Ordenando las observaciones de menor a mayor; es decir, los datos brutos ordenados como $x_{(1)}\leq x_{(2)}\leq\cdots\leq x_{(n)}$, donde los valores entre paréntesis denotan el número de rango de la observación. Entonces, si el número $n$ de observaciones es impar, la mediana se define como:

$$ x_{Med} = x_{\frac{n+1}{2}} $$

Si $n$ es par, la mediana se define como el promedio de las observaciones en el medio de la lista ordenada

$$ x_{Med} = \dfrac{x_{\frac{n}{2}}+ x_{\frac{n}{2}+1}}{2} $$

Finalmente, la moda se define como la observación que se encuentra con mayor frecuencia en el conjunto de datos; es decir,

$$ x_{Mod}:\text{ Valor con mayor frecuencia}$$

Si una distribución tiene una sola moda y es simétrica, contiene ese $\bar{x} = x_{Med} = x_{Mod} $. Si está sesgada a la izquierda (derecha), contiene $\bar{x}\leq x_{Med}\leq x_{Mod}$ ($\bar{x}\geq x_{Med}\geq x_{Mod}$).

Una expresión más general para la mediana es un cuantil. Un $p-$cuantil $x_{p}$ con $0<p<1$, se define como el valor para el cual:

+ Al menos una proporción $p$ de los valores de la muestra es menor o igual que $x_{p}$.
+ Al menos una proporción $1-p$ de los valores de la muestra es mayor o igual que $x_{p}$.

Es decir,

$$ \dfrac{\text{number}(x-\text{values})\leq x_{p}}{n}\geq p $$

y

$$ \dfrac{\text{number}(x-\text{values}
)\geq x_{p}}{n}\geq 1-p $$

Los cuantiles especiales son:

+ $x_{0.5}$: mediana
+ $x_{0.25}, \ \ \ x_{0.75}$: cuartiles inferior y superior
+ $x_{0.1}, x_{0.2},\cdots,x_{0.9}$: deciles

In [ ]:
mortgage_data.FICO_orig_time.describe() #estadística descriptiva

In [ ]:
for q in [0.01, 0.25, 0.5, 0.75, 0.99]: #cuantiles
    print(f"{q} cuantil: {mortgage_data.FICO_orig_time.quantile(q = q)}")

In [ ]:
print(f"moda: {mortgage_data.FICO_orig_time.mode().values}")
print(f"media: {mortgage_data.FICO_orig_time.mean()}")
print(f"mediana: {mortgage_data.FICO_orig_time.quantile(q=0.5)}")

In [ ]:
plt.subplots()
plt.hist(mortgage_data.FICO_orig_time, bins=50, density = True)
plt.axvline(x = mortgage_data.FICO_orig_time.mean(), color = 'r', label = "media")
plt.axvline(x = mortgage_data.FICO_orig_time.quantile(q=0.5), color = "k", label = 'mediana')
plt.title('FICO_orig_time histogram')
plt.legend()
plt.show()

Los cuantiles se pueden usar para una comparación gráfica con distribuciones estándar, como una distribución normal. Se puede crear un diagrama cuantil-cuantil usando la función `probplot` de` scipy.stats`, que compara para cada valor su valor cuantil con el teórico bajo una distribución específica.

La biblioteca SciPy, es una colección de algoritmos numéricos y cajas de herramientas específicas de dominio, que incluye procesamiento de señales, optimización, estadísticas y mucho más. En este cuaderno, se utiliza el modulo estadístico `stats`.

https://docs.scipy.org/doc/scipy/reference/stats.html

In [ ]:
import scipy.stats as stats

##QQplot
fig, ax = plt.subplots() #para este gráfico utilizo la interfaz orientada a objetos de mpl
stats.probplot(mortgage_data.FICO_orig_time.values, plot = ax, dist = "norm")

Si los datos fueran de una distribución normal, tanto los cuantiles empíricos como los teóricos deberían ser aprodimadamente iguales y estar en la línea diagonal. Vemos divergencias para FICO, particularmente para observaciones extremas, lo que indica que los datos empíricos tienen colas diferentes que la distribución normal teórica.



---
## Medidas de dispersión

El intervalo o rango es simple, la diferencia entre los valores mínimo y máximo. Si consideramos el conjunto de datos ordenado, entonces:

$$ sp = x_{(n)} - x_{(1)} $$

El error cuadrático medio (MSE), la varianza de la muestra y la desviación estándar se definen como:

$$ \begin{array}{c}
MSE = \dfrac{1}{n}\cdot \sum\limits_{i=1}^{n} (x_{i}-\bar{x})^{2} \\
s^{2} = \dfrac{1}{n-1}\cdot\sum\limits_{i=1}^{n}(x_{i}-\bar{x})^{2} \\
s = +\sqrt{s^{2}}
\end{array}
$$

Todas las medidas calculan una distancia cuadrática promedio desde la media (o una raíz cuadrada de la misma). Para grandes $n$, las diferencias entre las primeras dos medidas se vuelven insignificantes y ambos números casi coinciden. La desviación estándar posee las mismas unidades que los datos. Las tres medidas dependen de la escala. En otras palabras, una variable con valores más altos en general debería exhibir una dispersión más alta, siendo todo lo demás igual. Para controlar esto, se puede estandariza y calcular el coeficiente de variación (CV), que se define como:

$$ \nu = \dfrac{s}{\bar{x}}$$

In [ ]:
variance = mortgage_data.FICO_orig_time.var(ddof = 1)
std_dev = np.sqrt(variance)
mse = mortgage_data.FICO_orig_time.var(ddof = 0)
coef_variation = std_dev / mortgage_data.FICO_orig_time.mean()

In [ ]:
print(f"Varianza: {round(variance,3)}")
print(f"Desviación estándar: {round(std_dev,3)}")
print(f"Coeficiente de variaciónn: {round(coef_variation,3)}")

___
## Medidas de asimetría y curtosis - métodos descriptivos

A continuación, calculamos medidas para la forma de la distribución: asimetría y curtosis. Primero, definimos el valor estandarizado para cada variable como:

$$ z_{i} = \dfrac{x_{i}-\bar{x}}{s} $$

Entonces la asimetría se calcula como

$$ skew = \dfrac{1}{n}\sum\limits_{i=1}^{n} z_{i}^{3} $$

Del mismo modo, la curtosis se calcula como:

$$ kurt = \dfrac{1}{n}\sum\limits_{i=1}^{n}z_{i}^{4} - 3$$

Tenga en cuenta que la media y la varianza de los valores estandarizados son $0$ y $1$, respectivamente. Un valor negativo de asimetría muestra que la distribución está sesgada hacia la izquierda; un valor positivo muestra que hay un sesgo a la derecha. La curtosis mide el pico de la distribución. Cuando resta el valor $3$, a veces se denomina exceso de curtosis ya que el valor se contrasta con el valor de $3$ para la distribución normal.

In [ ]:
skewness = stats.skew(mortgage_data.FICO_orig_time.values)
kurtosis = stats.kurtosis(mortgage_data.FICO_orig_time.values, fisher = True)

In [ ]:
print(f"Asimetría: {round(skewness,3)}")
print(f"Exceso de curtosis: {round(kurtosis,3)}")

---
LLegado este punto, se introduce un método para dataframes MUY útil. El método `describe()` provee información estadística del conjunto de datos de manera simple.

In [ ]:
mortgage_data.describe()

Otro método útil es `info()`, que provee información sobre el tipo de datos encontrados en cada columna, y de la memoria ocupada por el conjunto de datos.

In [ ]:
mortgage_data.info()

___
## Análisis bidimensional

### Distribuciones empíricas conjuntas

La distribución empírica conjunta calcula simultáneamente la distribución de frecuencia de dos o más variables. Las frecuencias absolutas conjuntas (análogamente relativas) se pueden resumir en una tabla de frecuencias bidireccionales de manera que:

$$ \begin{array}{c|ccc|c}
 & b_{1} & \cdots & b_{l} & \\
\hline
a_{1} & h_{11} & \cdots & h_{1l} & h_{1\cdot} \\
\vdots & \vdots & \ddots & \vdots & \vdots \\
a_{k} & h_{k1} & \cdots & h_{kl} & h_{k\cdot} \\
\hline
 & h_{\cdot 1} & \cdots & h_{\cdot l} & n 
\end{array}
$$

donde:
+ $a_{1},\cdots,a_{k}$:  valores / categorías de atributos de la variable $X$.
+ $b_{1},\cdots, b_{l}$:  valores / categorías de atributos de la variable $Y$.
+ $h_{ij} = h(a_{i},b_{j})$:  frecuencias absolutas conjuntas
+ $h_{i\cdot} = \sum\limits_{i=1}^{l}h_{ij}$:  frecuencias marginales absolutas de $X$.

In [ ]:
# en pandas esto se logra mediante la función crosstab
# se utilizan la variables default_time y FICO_orig_time, para FICO divido los datos en quintiles
# el caracter de barra permite continuar la línea debajo para mejorar la legibilidad
print("Absolutas")
pd.crosstab(index = mortgage_data.default_time, columns = pd.qcut(mortgage_data.FICO_orig_time,5\
                                    , labels = [0,1,2,3,4]), margins=True, margins_name='Total')

In [ ]:
print("Relativas") # el argumento normalize = True me permite calcular de forma relativa
pd.crosstab(index = mortgage_data.default_time, columns = pd.qcut(mortgage_data.FICO_orig_time,\
5, labels = [0,1,2,3,4]),margins = True, margins_name = 'Total', normalize =True)

Otra forma de inferir la relación entre ambas variables es mirar los diagrama de cajas. Un diagrama de caja consiste en una caja y bigotes.

In [ ]:
fig, ax = plt.subplots() #interfaz orientada a objetos
sn.boxplot(x = "default_time", y = "FICO_orig_time", data = mortgage_data, ax = ax)#trazado mediante etiquetas
plt.show()

Los cuadros muestran los cuartiles del conjunto de datos, mientras que los bigotes se extienden para mostrar el resto de la distribución (es decir, el máximo y el mínimo), excepto los puntos que se determinan como _valores atípicos_ utilizando el test de vallas de Tukey (valores a distancias del cuartil más cercano mayores de 1.5 veces la distancia intercuartil, se consideran atípicos).

In [ ]:
fig, ax = plt.subplots() # para otro conjunto de variables
sn.boxplot(x = 'default_time', y = 'LTV_orig_time', data = mortgage_data, ax = ax)
plt.show()

La ubicación de la caja es más alta para FICO y más baja para LTV (loan to value) en la categoría de `no default` en comparación con la categoría `default`, lo que muestra cierta interrelación entre las variables en el sentido de que los puntajes FICO más altos corresponden a una frecuencia de `default` más baja y los valores LTV más altos corresponden a mayor frecuencia de `default`.

Para hacer declaraciones más fuertes sobre la dependencia, uno necesita calcular medidas numéricas. Para comenzar, primero se necesita un punto de referencia, que obviamente es el caso de la independencia. Estadísticamente, dos variables son empíricamente independientes si las frecuencias conjuntas en la tabla de dos vías están dadas por el producto de las respectivas frecuencias marginales en toda la tabla; es decir,

$$
f_{ij} = \dfrac{h_{i}h_{j}}{n^{2}}
$$

$$
h_{ij} = \dfrac{h_{i}h_{j}}{n}
$$

Una vez definido un punto de referencia, ahora podemos comenzar a hacer evaluaciones sobre la dirección y la fuerza de la dependencia midiendo la desviación de la independencia. Esto se realiza a través de medidas de correlación y dependencia.

___
## Medidas de correlación

### Tabla de contingencia y prueba de chi-cuadrado

La tabla de contingencia contiene las frecuencias $h_{ij}$ de todas las combinaciones de resultados, cuando hay múltiples variables aleatorias que pueden tomar un número finito de valores. Bajo la hipótesis nula de independencia, podemos calcular las _frecuencias esperadas_ $\frac{h_{i}h_{j}}{n}$, en base a las sumas marginales. El estadístico $\chi^{2}$, por definición, es:

$$ 
\chi = \sum\limits_{i,j}\dfrac{\left(h_{ij}-\frac{h_{i}h_{j}}{n}\right)^{2}}{\frac{h_{i}h_{j}}{n}}
$$

Cuando hay suficientes observaciones, esta variable sigue aproximadamente la distribución de chi cuadrado (la distribución de la suma de las variables normales al cuadrado). Una vez que obtenemos el $p-value$, podemos rechazar o aceptar la hipótesis nula de independencia. Entonces, podemos concluir (o no) que existe una correlación significativa entre las variables.

`scipy` implementa esta prueba mediante ` scipy.stats.chi2_contingency() `, que devuelve varios objetos. Estamos interesados en el segundo resultado, que es el $p-value$.

In [ ]:
from scipy.stats import chi2_contingency

cont_table = pd.crosstab(index = mortgage_data.default_time, columns = pd.qcut(\
                            mortgage_data.FICO_orig_time,5, labels = [0,1,2,3,4])) #utilizo otra vez crosstab()
chi2, p, dof, ex = chi2_contingency(cont_table.values) #values me devuelve un array con los valores
#noten que chi2_contingency posee varios argumentos de salida

In [ ]:
print(f'Chi cuadrado = {chi2}')
print(f'p-value = {p}')


El $p-value$ es cero, por lo que rechazamos la hipótesis nula y concluimos que existe una correlación estadísticamente significativa entre las variables.

Para datos métricos podemos usar la covarianza muestral, definida como

$$
s_{xy} = \dfrac{1}{n-1}\sum\limits_{i=1}^{n} (x_{i}-\bar{x})(y_{i}-\bar{y})
$$

La _covarianza muestral_ depende de la escala. Una medida estandarizada es la correlación muestral definida como:

$$ 
r_{xy} = \dfrac{s_{xy}}{s_{x}s_{y}}
$$

A veces se la denomina _correlación de Bravais-Pearson_ y adquiere valores $-1\leq r_{xy}\leq +1$. Es una medida para la dependencia _lineal_. Un coeficiente de $r_{xy} \approx 0$ no significa necesariamente que las variables no sean dependientes, solo que no hay una relación lineal.

In [ ]:
from scipy.stats import pearsonr #importo el método personr
covar = np.cov(mortgage_data['FICO_orig_time'], mortgage_data['LTV_orig_time'])#utilizo la covarianza de numpy
r_pearson = pearsonr(mortgage_data['FICO_orig_time'], mortgage_data['LTV_orig_time'])# correlación de Pearson
print(f'Covarianza: {round(covar[0,1],2)}')
print(f'Coeficiente de correlación de Bravais-Pearson: {round(r_pearson[0],2)}')

La dependencia también se puede representar gráficamente utilizando un diagrama de dispersión.

In [ ]:
inds = np.random.choice(len(mortgage_data['FICO_orig_time']), 5000) #elijo 5000 valores FICO al azar
x = mortgage_data['FICO_orig_time'].iloc[inds].values # defino x como esos valores
y = mortgage_data['LTV_orig_time'].iloc[inds] # defino y como los valores LTV asociados

plt.subplots()
plt.scatter(x,y, s = 20, edgecolor = 'k', alpha = 0.6)
plt.xlabel('LTV_orig_time')
plt.ylabel('FICO_orig_time')

___
## Intervalos de confianza

Sea $\theta$  el parámetro (desconocido) en la población de interés y sea $\hat{\theta}$ su estimación desde una muestra aleatoria. Luego construimos los límites inferior y superior $L_{i}$ y $L_{s}$ de manera que:

$$
P(L_{i}\leq \theta\leq L_{s}) = 1-\alpha
$$

donde $\alpha$ se llama probabilidad de error y $1-\alpha$ es el nivel de confianza.

Muchos modelos resultan en estimadores de parámetros distribuidos aproximadamente normalmente. Entonces, los intervalos de confianza se pueden construir fácilmente. Pensemos en un estimador $\hat{\theta}$ que se distribuye normalmente con expectativa $\theta$ y varianza $\sigma_{\theta}^{2}$. Entonces $\frac{\hat{\theta}-\theta}{\sigma_{\theta}}$ se encuentra estandarizado y normalmente distribuido, y debido a las propiedades de la distribución normal estándar obtenemos

$$ 
P(-z_{1-\alpha/2} \leq \frac{\hat{\theta}-\theta}{\sigma_{\theta}}\leq z_{1-\alpha/2})=1-\alpha,
$$

donde $z_{1-\alpha/2}$ es el percentil $1-\alpha/2$ de la distribución normal estándar. Reordenando :

$$
P(\hat{\theta}-z_{1-\alpha/2}\cdot\sigma_{\theta}\leq\theta\leq\hat{\theta}+z_{1-\alpha/2}\cdot\sigma_{\theta})=1-\alpha
$$

Esto da el intervalo de confianza

$$ 
\left[\hat{\theta}-z_{1-\alpha/2}\cdot\sigma_{\theta}; \ \ \ \hat{\theta}+z_{1-\alpha/2}\cdot\sigma_{\theta} \right]
$$

Por lo general, sin embargo, la desviación estándar del estimador también es desconocida y debe estimarse a partir de la muestra. Sea $\hat{\sigma}_{\theta}$ la estimación. Entonces el intervalo de confianza se convierte en:

$$
\left[ \hat{\theta}-t_{n-1,1-\alpha/2}\cdot\hat{\sigma}_{\theta}; \ \ \ \hat{\theta}+t_{n-1,1-\alpha/2}\cdot\hat{\sigma}_{\theta}\right]
$$

donde $t_{n-1,1-\alpha/2}$ es el percentil $1-\alpha/2$ de la distribución $t$ de Student con $n-1$ grados de libertad y $n$ es el tamaño de la muestra.

In [ ]:
from scipy.stats import t

n = int(0.6*len(mortgage_data)) #%60 del total de datos
alpha = 0.99 
sample = np.random.choice(mortgage_data['FICO_orig_time'].values, n) #elijo los valores al azar 
mean_value = np.mean(sample) #calculo la media
std_value = np.std(sample) #calculo la desviación estándar
h = std_value * t.ppf(1-0.5*alpha, n-1) # el error
lower_bound = mean_value - h # límite inferior
upper_bound = mean_value + h # límite superior
print(f'Media: {round(mean_value,3)}')
print(f'Intervalo de confianza: [{round(lower_bound,3)};{round(upper_bound,3)}]')

Existen muchas otras herramientas y técnicas para explorar conjuntos de datos. En este cuaderno se revisar las más comunes y útiles. Queda pendiente la presentación y ejemplos de otros tests estadísticos. 

Una vez terminado el análisis exploratorio, se procede a preprocesar los datos. Las técnicas de preprocesamiento a utilizar dependen del modelo con el que se quiere alimentar los datos.